In [1]:
import sys
!{sys.executable} -m pip install xgboost
import xgboost as xgb
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
print("First we fill in na numerical data and drop na nonnumerical data")
print(train.columns[train.isna().any()])
train['LotFrontage'] = train['LotFrontage'].fillna(train['LotFrontage'].mean())
train['MasVnrArea'] = train['MasVnrArea'].fillna(train['MasVnrArea'].mean())
train['GarageYrBlt'] = train['GarageYrBlt'].fillna(train['GarageYrBlt'].mean())
test['LotFrontage'] = test['LotFrontage'].fillna(test['LotFrontage'].mean())
test['MasVnrArea'] = test['MasVnrArea'].fillna(test['MasVnrArea'].mean())
test['GarageYrBlt'] = test['GarageYrBlt'].fillna(test['GarageYrBlt'].mean())
train_empty = train.columns[train.isna().any()]
test_empty = test.columns[test.isna().any()]
X = train.drop(columns = ['SalePrice', 'Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature'])
y = train['SalePrice']
X_true = test.drop(columns= [ 'Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature'])
print(X_true.shape, X.shape, y.shape)
print(X_true.columns)
one_hot_encoded_training_predictors = pd.get_dummies(X)
one_hot_encoded_test_predictors = pd.get_dummies(X_true)
X, X_true = one_hot_encoded_training_predictors.align(one_hot_encoded_test_predictors, join='inner', axis=1)
X.shape, X_true.shape

First we fill in na numerical data and drop na nonnumerical data
Index([u'LotFrontage', u'Alley', u'MasVnrType', u'MasVnrArea', u'BsmtQual',
       u'BsmtCond', u'BsmtExposure', u'BsmtFinType1', u'BsmtFinType2',
       u'Electrical', u'FireplaceQu', u'GarageType', u'GarageYrBlt',
       u'GarageFinish', u'GarageQual', u'GarageCond', u'PoolQC', u'Fence',
       u'MiscFeature'],
      dtype='object')
((1459, 64), (1460, 64), (1460L,))
Index([u'Id', u'MSSubClass', u'MSZoning', u'LotFrontage', u'LotArea',
       u'Street', u'LotShape', u'LandContour', u'Utilities', u'LotConfig',
       u'LandSlope', u'Neighborhood', u'Condition1', u'Condition2',
       u'BldgType', u'HouseStyle', u'OverallQual', u'OverallCond',
       u'YearBuilt', u'YearRemodAdd', u'RoofStyle', u'RoofMatl',
       u'Exterior1st', u'Exterior2nd', u'MasVnrArea', u'ExterQual',
       u'ExterCond', u'Foundation', u'BsmtFinSF1', u'BsmtFinSF2', u'BsmtUnfSF',
       u'TotalBsmtSF', u'Heating', u'HeatingQC', u'CentralAir', u'1stF

((1460, 205), (1459, 205))

In [10]:
# X_train, X_2, y_train, y_2 = train_test_split(X, y, test_size=.15, random_state=9000)
# X_train.shape, y_train.shape, X_2.shape, y_2.shape

((1241, 64), (1241L,), (219, 64), (219L,))

In [3]:
kf = KFold(n_splits = 5, random_state = 4)

In [4]:
imputer = SimpleImputer()
rb = RobustScaler()
xgbr = xgb.XGBRegressor()
pipe = Pipeline([('imputer', imputer), ('rb', rb), ('xgbr', xgbr)])
pipe.get_params().keys()

['xgbr__subsample',
 'xgbr__min_child_weight',
 'xgbr__gamma',
 'xgbr__max_depth',
 'rb__copy',
 'xgbr__colsample_bytree',
 'xgbr__booster',
 'xgbr__colsample_bylevel',
 'xgbr__max_delta_step',
 'xgbr',
 'xgbr__random_state',
 'imputer',
 'imputer__verbose',
 'xgbr__reg_alpha',
 'rb__quantile_range',
 'xgbr__scale_pos_weight',
 'rb',
 'memory',
 'xgbr__seed',
 'xgbr__silent',
 'rb__with_centering',
 'xgbr__base_score',
 'imputer__fill_value',
 'rb__with_scaling',
 'xgbr__n_jobs',
 'xgbr__objective',
 'xgbr__reg_lambda',
 'xgbr__nthread',
 'xgbr__learning_rate',
 'imputer__missing_values',
 'xgbr__missing',
 'imputer__strategy',
 'steps',
 'imputer__copy',
 'xgbr__n_estimators']

In [5]:
param_grid = {'imputer__strategy' : ['median', 'mean'],
              'xgbr__n_estimators' : [800, 1000, 1200],
              'xgbr__learning_rate' : [0.03, 0.04, 0.05]
               }
gs = GridSearchCV(pipe, param_grid=param_grid, cv=kf)
gs.fit(X, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=4, shuffle=False),
       error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('imputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
       verbose=0)), ('rb', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('xgbr', XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel...
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'imputer__strategy': ['median', 'mean'], 'xgbr__learning_rate': [0.03, 0.04, 0.05], 'xgbr__n_estimators': [800, 1000, 1200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [7]:
model = gs.best_estimator_
model.fit(X, y)
pred = model.predict(X_true)

In [ ]:
mse_scores = -cross_val_score(model, X, y, cv=kf, scoring='neg_mean_squared_error')
rmse_scores = mse_scores**0.5

In [ ]:
print(pred)
print(mse_scores, rmse_scores)

In [ ]:
rmse_scores.mean(), rmse_scores.std()

In [ ]:
predicted_prices = model.predict(X_true)

In [ ]:
true_prices = pd.read_csv('sample_submission.csv')
true_prices.SalePrice

In [ ]:
len(predicted_prices)

In [ ]:
X_true

In [ ]:
true_prices.SalePrice - X_true